In [15]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import polars as pl
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from tqdm import tqdm


In [16]:
driver = webdriver.Chrome("C:\\Users\\franc\\Downloads\\chromedriver_win32\\chromedriver.exe")



C:\Users\franc\AppData\Local\Temp\ipykernel_21232\1631663030.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\\Users\\franc\\Downloads\\chromedriver_win32\\chromedriver.exe")


In [17]:
LINK = 'https://www.remax.pt/arrendar?searchQueryState=%7B"regionName":"","sort":%7B"fieldToSort":"ContractDate","order":1%7D,"businessType":2,"page":{i},"t":"","mapIsOpen":false,"mapScroll":false,"searchNextToMe":false,"rooms":-1,"price":null%7D'

In [18]:
def make_listing(price,district,city,property_type,area,bathrooms,bedrooms):
    return {
        'Price': price,
        'District':district,
        'City':city,
        'Type':property_type,
        'Area':area,
        'Number Bathrooms': bathrooms,
        'Number Bedrooms':bedrooms
    }

In [19]:

all_listings = []
for i in tqdm(range(1,101),total=100):
    driver.get(LINK.format(i=i))
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div/div/div[1]/div/div[2]/div/div[2]/div[1]/div/div/h1')))
    
    for l in driver.find_elements(By.XPATH,'//*[@class="listing-search-searchdetails-component"]/a/div/div'):
        price = int(l.find_element(By.CLASS_NAME,'listing-price').text.split('€')[0].replace(" ",""))
        address = l.find_element(By.CLASS_NAME,'listing-address').text
        main = address.split(',')[0].split('-')
        district = main[0].strip()
        city = main[1].replace(u'\xa0', u'').strip()
        property_type = l.find_element(By.CLASS_NAME,'listing-type').text
        try:
            area = int(l.find_element(By.CLASS_NAME,'listing-area').text.replace(' m\n2',""))
        except:
            area = None
        try:
            bathrooms = int(l.find_element(By.CLASS_NAME,'listing-bathroom').text)
        except:
            bathrooms = None
        try:
            bedrooms = int(l.find_element(By.CLASS_NAME,'listing-bedroom').text)
        except:
            bedrooms = None
        all_listings.append(make_listing(price,district,city,property_type,area,bathrooms,bedrooms))
        
    
        
    
    

 31%|███       | 31/100 [01:17<02:46,  2.41s/it]

In [ ]:
data = pl.DataFrame(all_listings)
data.write_parquet('ScrappedData.parquet')

NameError: name 'all_listings' is not defined

In [ ]:
list(data['Type'].unique())

['Outros - Comercial',
 'Terreno',
 'Outros - Habitação',
 'Investimentos',
 'Escritório',
 'Duplex',
 'Armazém',
 'Quinta',
 'Trespasse',
 'Industrial',
 'Moradia',
 'Prédio',
 'Herdade',
 'Garagem',
 'Apartamento',
 'Loja']

In [ ]:
data.filter(data['Type'].is_in([
 'Outros - Habitação',
 'Investimentos',
 'Escritório',
 'Duplex',
 'Armazém',
 'Quinta',
 'Trespasse',
 'Industrial',
 'Moradia',
 'Prédio',
 'Herdade',
 'Apartamento']))\
.groupby('District')\
    .agg((pl.col('Price').sum()/pl.col('Area').sum())\
         .alias('Mean Price per Sqr Meter Per District'),
         pl.col('Price').mean().alias('Mean Price'),
        )\
    .sort('Mean Price per Sqr Meter Per District',descending=True)

District,Mean Price per Sqr Meter Per District,Mean Price
str,f64,f64
"""Lisboa""",18.040116,2813.958472
"""Ilha da Madeir...",11.0,893.75
"""Faro""",9.944399,1609.666667
"""Setúbal""",9.767604,1596.684783
"""Porto""",9.700449,1586.952128
"""Ilha de São Mi...",8.402904,1322.857143
"""Guarda""",7.142857,500.0
"""Coimbra""",6.527397,744.53125
"""Vila Real""",6.19469,875.0
